In [1]:
import numpy as np
import cv2
from PIL import Image    
import matplotlib.pyplot as plt   
import matplotlib.image as mpimg
from matplotlib.colors import NoNorm
from skimage import transform
from glob import glob
import imageio
%matplotlib inline

In [2]:
from collections import deque
import os
import os.path
import random
import string

def random_renames(files):
    """
    Generate a list of randomised renames to shuffle a list of files.

    Example:
        random_renames(["a", "b", "c"])
    This might generate:
        ("a", "c"), ("b", "a"), ("c", "b")

    Where each tuple indicates first the original name, and then the new name
    of the file.
    """

    remaining_names = files[:]  # Note: copying the list because we will modify this one

    for orig in files:
        # Find a list of potential new names, which is all unused names, except
        # the current filename
        potential_names = [name for name in remaining_names if name != orig]
        # Pick one from that list and remove it from our pool of unused names
        newname = random.choice(potential_names)
        remaining_names.remove(newname)

        # Output this rename from the generator
        yield (orig, newname)


def main(path):
    """
    Renames files in the 'shuffleme' directory in such a way as to shuffle all
    of them between their existing names.
    """
    # I just created a directory called "shuffleme" in CWD, replace this with
    # your full path if necessary
    path = path

    # Get the existing files from that directory
    files = os.listdir(path)

    # Prefix all the names with the dir name
    files = [os.path.join(path, name) for name in files]

    # Generate our list of randomised renames from the directory list.  This is
    # a deque for reasons that might be clearer later on.  (A deque is like a
    # list but designed for fast adding/removing from either end)
    file_renames = deque(random_renames(files))

    while file_renames:  # While there are still files left to rename
        # Get our first intended rename
        oldname, newname = file_renames.popleft()

        # Avoid trying to rename files to the same name. This involves adding
        # random characters to the name until we hit on a name that doesn't
        # exist, then renaming to this compromise of a name, and then delaying
        # the rename to the real name.
        tempname = newname
        while os.path.exists(tempname):
            tempname += random.choice(string.ascii_letters)

        # If we did have to change names...
        if tempname != newname:
            # ... then remember to rename to the new name
            file_renames.append((tempname, newname))
            # Sanity check: we definitely *are* going to rename the original
            # user of this name away, right? If not, we're in for an infinite
            # loop.
            assert any(name == newname for name, _ in file_renames)

        # Perform the actual rename. Not renaming to newname (actual new name),
        # but tempname (same as newname unless that file exists, in which case
        # we already delayed a separate rename).
        os.rename(oldname, tempname)

In [3]:
# Function to rename all files 

def rename_imgs(file_dir, new_img_name): 
    i = 1      
    for ori_img_name in os.listdir(file_dir): 
        dst = new_img_name + "_" + str(i) + ".jpg"
        src = file_dir + "/" + ori_img_name 
        dst = file_dir + "/" + dst 
        os.rename(src, dst) 
        i += 1

In [4]:
def shuffle_rename(file_dir, new_img_name):
    main(file_dir)
    rename_imgs(file_dir, new_img_name)

In [5]:
def crop_center_square_and_resize(input_img):
    #pil_img = Image.open(input_img)
    pil_img = input_img
    crop_width, crop_height = min(pil_img.size), min(pil_img.size)
    img_width, img_height = pil_img.size
    left = (img_width - crop_width) // 2
    top = (img_height - crop_height) // 2
    right = (img_width + crop_width) // 2
    bottom = (img_height + crop_height) // 2
    cropped_img = pil_img.crop((left, top, right, bottom))
    resized_img = cropped_img.resize((640, 640))
    return resized_img

In [6]:
from PIL import Image, ImageOps

def fill_black(img): 
    #img = Image.open(img)
    img_with_border = ImageOps.expand(img,border=100,fill='black')
    #final_img = crop_center_square_and_resize(img_with_border)
    return img_with_border

In [7]:
def crop_and_resize_img(source_dir, target_dir):
    os.makedirs(target_dir+"/")
    jpgdata = glob(os.path.join(source_dir+"/","*.jpg"))
    jpgdata = np.sort(jpgdata)
    for i in range(0,jpgdata.size,1):
        image = Image.open(jpgdata[i])
        filled_img = fill_black(image)
        final_img = crop_center_square_and_resize(filled_img)
        if final_img.mode in ("RGBA", "P"):
            final_img = final_img.convert("RGB")
#         imageio.imwrite(target_dir+"/resized_{}".format(i)+".jpg", 
#                         final_img)
        final_img.save(target_dir+"/resized_{}".format(i)+".jpg")
    print("image processing is finished")

In [8]:
# file_dir = 'D:/image_datasets_deeplearning/kidney_ultrasound_imageSet'
# new_img_name = 'kidney_img'
# shuffle_rename(file_dir, new_img_name)

In [11]:
base_dir = "D:/OneDrive - 亞洲大學[Asia University](1)/Research/CS/pro_tasi/kidney/kidney_20200821T101308Z_001/datasets/images"
source_dir = base_dir + "/train"
target_dir = base_dir + "/train_resized"

In [12]:
crop_and_resize_img(source_dir, target_dir)

image processing is finished
